In [7]:
from app.configuration.config import TinvestSettings, PgSqlSettings
from datetime import datetime, timezone, timedelta

from tinkoff.invest import CandleInterval

from app.repositories.tinvest_candles import TinvestCandlesRepository
from app.services.tinvest import TinvestAdapter

In [8]:
tinvest_config = TinvestSettings(_env_file='../app/.env')
tinvest = TinvestAdapter(tinvest_config)

sql_config = PgSqlSettings(_env_file='../app/.env')
candles_repo = TinvestCandlesRepository(sql_config)

In [9]:
symbol = tinvest.get_instrument(ticker="SBER", class_code="TQBR")

end_date_utc = datetime.now(timezone.utc)
days_count = 10
start_date_utc = end_date_utc - timedelta(days=days_count)

candles = tinvest.get_candles(
    instrument_id=symbol.uid,
    start_date_utc=start_date_utc,
    end_date_utc=end_date_utc,
    interval=CandleInterval.CANDLE_INTERVAL_10_MIN)

In [10]:
candles_repo.insert_candles("SBER", CandleInterval.CANDLE_INTERVAL_10_MIN, candles)

607

In [11]:
df = candles_repo.get_candles("SBER", CandleInterval.CANDLE_INTERVAL_10_MIN)
df

,time_utc,interval,symbol,open,close,high,low,volume,is_completed,candle_source
0,2025-01-24 16:40:00+00:00,8,SBER,281.13,280.84,281.13,280.81,8287.0,True,1
1,2025-01-24 16:50:00+00:00,8,SBER,280.83,281.04,281.04,280.76,6811.0,True,1
2,2025-01-24 17:00:00+00:00,8,SBER,281.00,281.18,281.18,280.76,5663.0,True,1
3,2025-01-24 17:10:00+00:00,8,SBER,281.14,281.04,281.19,281.00,6755.0,True,1
4,2025-01-24 17:20:00+00:00,8,SBER,281.00,281.05,281.14,280.97,5422.0,True,1
...,...,...,...,...,...,...,...,...,...,...
602,2025-02-03 16:00:00+00:00,8,SBER,279.22,278.99,279.22,278.93,5082.0,True,1
603,2025-02-03 16:10:00+00:00,8,SBER,278.98,279.01,279.08,278.98,12349.0,True,1
604,2025-02-03 16:20:00+00:00,8,SBER,279.01,279.00,279.09,279.00,4429.0,True,1
605,2025-02-03 16:30:00+00:00,8,SBER,279.01,279.03,279.07,279.00,3890.0,True,1
